In [1]:
from mimetypes import init
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
from selenium.webdriver.common.keys import Keys
import json 

In [2]:
class Hotel_Tripadvisor():
    def __init__(self, link_hotel_ci):
        self.linkh= link_hotel_ci
        self.driver=  webdriver.Chrome("/home/henryx/prueba/web_scraping/chromedriver")
        self.url = "https://www.tripadvisor.com//"

    def clickon(self, xxpad):
        try:
            veras= self.driver.find_element("xpath", xxpad )
            veras.click()
            time.sleep(0.5)
            return True
        except:
                print('problema en click en  ' + xxpad)
                return False
    
   
    def scrooll(self):
        elemet= self.driver.find_element("tag name", "body")
        for i in range(4):
            elemet.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)
    

    def hotelesc(self ):
        try:
            self.driver.get(self.linkh)   
            self.clickon('//*[@id="component_7"]/div/button')
            self.scrooll()
            time.sleep(3)
            page = self.driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            eq = soup.find_all('div',class_="listing_title")  ## encuantra la lista de hoteles que hay que buscar
            return eq
        except:
            return False
    

    def inf_hotel(self, h_registrado, hotel):
        try:
            url_re = url + hotel.next.attrs['href']
            time.sleep(2)
            h_registrado["descripccion"]=hotel.text
            h_registrado["link"]=url_re
            ## ingresando al hotel
            self.driver.get(url_re)
            time.sleep(2)
           
            self.clickon('/html/body/div[11]/section/div/div/div[2]/div/div[1]/button')  # si existe el aununcio d etraducir
            
            time.sleep(2)
            page = self.driver.page_source
            soup = BeautifulSoup(page, 'html.parser') 
            h_registrado['nombre'] = soup.find('div',class_="_b8stb0").text
            h_registrado["precio"] = soup.find('div',class_="_ud8a1c").find('span',class_="_tyxjp1").text
            h_registrado["puntuacion_ge"] = soup.find('div',class_="_ud8a1c").find('span',class_="_12si43g").text
            
            time.sleep(0.5)
            self.scrooll()
            sera_c = self.clickon( '//*[@id="site-content"]/div/div[1]/div[4]/div/div/div/div[2]/section/div[3]/div/div/div[6]/div/div[2]/div[2]/button')
            sera_c = self.clickon('//*[@id="site-content"]/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span[1]/span[3]/button') if sera_c == False else False
            sera_c = self.clickon( '//*[@id="site-content"]/div/div[1]/div[4]/div/div/div/div[2]/section/div[4]/button') if sera_c == False else False
            return h_registrado
                
        except:
                pass

    def open_hidenc(self):
            time.sleep(1)
            self.driver.page_source
            time.sleep(1)

            try:
                elemet= self.driver.find_element("tag name", "body")
                letm= self.driver.find_element("xpath",'//*[@id="reviews-search-input"]')
                letm.click()
                time.sleep(0.5)
                for i in range(4):
                    elemet.send_keys(Keys.PAGE_DOWN)
                    time.sleep(1)
            except:
                pass

    def get_comments(self):
      
        try:
            hcomen=[]
            page = self.driver.page_source
            soup = BeautifulSoup(page, 'html.parser')

            comentarios = soup.find_all('div',class_="r1are2x1 dir dir-ltr")
            for comen in comentarios:
                incommet = dict()
                incommet['comentario'] = comen.find('span', class_= 'll4r2nl dir dir-ltr').text
                incommet['fechac'] = comen.find('ol', class_= '_7h1p0g').text
                hcomen = hcomen + [incommet]
            return hcomen
        except:
            print('algo pasa con ' )

    def ingest(self):
        resultado= []
        eq = self.hotelesc()
        for hotel in eq:
            h_registrado = {}
            h_registrado = self.inf_hotel( h_registrado, hotel)
            self.open_hidenc()
            hcomen = self.get_comments()
            h_registrado['comentarios']= hcomen
            resultado= resultado + [h_registrado]
        print('hola estoy aqui')
        json_object = json.dumps(resultado)
        with open("airbnb/sample-airbnb.json", "w") as outfile:
            outfile.write(json_object)


urlprueba= "https://www.tripadvisor.com/Hotels-g677335-Ambato_Tungurahua_Province-Hotels.html"
Hotel_Tripadvisor(urlprueba).ingest()


/tmp/ipykernel_593217/274911290.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver=  webdriver.Chrome("./chromedriver")


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


/tmp/ipykernel_592235/2257337969.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver=  webdriver.Chrome("./chromedriver")


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home
